# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
# load "cities.csv" into a dataframe
df = pd.read_csv("cities.csv")
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,41.09,64,93,4.97,AU,1633291202
1,Barrow,71.2906,-156.7887,30.22,80,90,12.66,US,1633291187
2,Nizhniy Kuranakh,58.8353,125.4869,19.71,97,38,3.33,RU,1633291208
3,Albany,42.6001,-73.9662,66.09,88,100,1.30,US,1633290937
4,Teberda,43.4456,41.7448,48.06,97,100,4.47,RU,1633291212


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

# Store humidity data # and convert to float
humidity = df["Humidity"]#.astype(float)

In [33]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=250,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,41.09,64,93,4.97,AU,1633291202
1,Barrow,71.2906,-156.7887,30.22,80,90,12.66,US,1633291187
2,Nizhniy Kuranakh,58.8353,125.4869,19.71,97,38,3.33,RU,1633291208
3,Albany,42.6001,-73.9662,66.09,88,100,1.30,US,1633290937
4,Teberda,43.4456,41.7448,48.06,97,100,4.47,RU,1633291212


In [103]:
# Narrow down the cities to fit the following conditions:
# max temperature in the range of 70 - 80
# Wind speed < 10 mph
# Cloudiness = 0
cities_df = df.loc[(df['Max Temp']>70) & (df['Max Temp']<80) & (df['Wind Speed']<10) & (df['Cloudiness'] ==0), :]
hotel_df = cities_df[["City", "Country", "Lat", "Lng"]]
hotel_df

,City,Country,Lat,Lng
221,Wajima,JP,37.4000,136.9000
342,Gweta,BW,-20.1833,25.2333
400,Jalu,LY,29.0331,21.5482
425,Elko,US,41.0002,-115.5012
458,Nishiwaki,JP,34.9833,134.9667
467,Misratah,LY,32.3754,15.0925
496,San Juan,AR,-31.5375,-68.5364


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [104]:
# set up a parameters dictionary that will be updated with new city each iteration
target_radius = 5000
target_type = "hotel"
params = {
#     "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through the hotel_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']

    # update address key value
    params['location'] = f"{lat},{lng}"
    try:
        # make request
        hotels = requests.get(base_url, params=params)
    
        # convert to json
        hotels = hotels.json()
        
        # store hotel name back to the hotel_df dataframe
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][1]['name']
        
        # Replace the lat and lng in the dataframe to the hotels location
        hotel_df.loc[index, "Lat"] = hotels['results'][1]['geometry']['location']['lat']
        hotel_df.loc[index, "Lng"] = hotels['results'][1]['geometry']['location']['lng']
        print(f"{row['City']} has more than 1 hotels.")
        
    except:
        print(f"{row['City']} has no hotels in 5000m range.")
# Visualize to confirm lat lng appear
hotel_df

C:\Users\fyang\anaconda3\envs\PythonData38\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\fyang\anaconda3\envs\PythonData38\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Wajima has more than 1 hotels.
Gweta has more than 1 hotels.
Jalu has more than 1 hotels.
Elko has no hotels in 5000m range.
Nishiwaki has more than 1 hotels.
Misratah has more than 1 hotels.
San Juan has more than 1 hotels.


,City,Country,Lat,Lng,Hotel Name
221,Wajima,JP,37.401661,136.896323,Wajiima Onsen Yashio
342,Gweta,BW,-20.209549,25.257874,Gweta Lodge
400,Jalu,LY,29.041907,21.530543,fantry Kitchens
425,Elko,US,41.000200,-115.501200,NaN
458,Nishiwaki,JP,34.985998,134.979246,かおるちゃんラーメン
467,Misratah,LY,32.360212,15.068530,Ben Amer Center for the memorization of the Koran
496,San Juan,AR,-31.537125,-68.523943,Hotel Provincial


In [105]:
# remove the city that doesn't have a hotel in 5000m range
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
221,Wajima,JP,37.401661,136.896323,Wajiima Onsen Yashio
342,Gweta,BW,-20.209549,25.257874,Gweta Lodge
400,Jalu,LY,29.041907,21.530543,fantry Kitchens
458,Nishiwaki,JP,34.985998,134.979246,かおるちゃんラーメン
467,Misratah,LY,32.360212,15.068530,Ben Amer Center for the memorization of the Koran
496,San Juan,AR,-31.537125,-68.523943,Hotel Provincial


In [106]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [109]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))